# Dynasty Market Trends Analysis

**Purpose**: Analyze Keep Trade Cut (KTC) valuation trends over time to identify market inefficiencies and optimal buy/sell windows

**Author**: FF Analytics Team

**Date**: 2024-11-08

## Objectives

- Track KTC value changes over time for key players
- Identify players with rising/falling market values
- Compare positional value trends (RB decline curve, WR stability)
- Find potential buy-low and sell-high candidates

In [ ]:
# Setup: Imports and configuration
import os
from pathlib import Path
from datetime import datetime, timedelta

import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (14, 7)
plt.rcParams["font.size"] = 10

In [ ]:
# DuckDB connection setup
external_root = os.environ.get("EXTERNAL_ROOT", str(Path.cwd().parent.parent / "data" / "raw"))

db_path = Path.cwd().parent.parent / "dbt" / "ff_data_transform" / "target" / "dev.duckdb"

if db_path.exists():
    conn = duckdb.connect(str(db_path), read_only=True)
    print(f"Connected to: {db_path}")
else:
    conn = duckdb.connect()  # In-memory
    print("Using in-memory database")

# Test connection
conn.execute("SELECT 1 AS test").fetchdf()

---
**📊 Data Freshness Check**

Last updated:
- KTC Valuations: 2024-11-07 (1 day old)
- NFLverse Stats: 2024-11-08 (0 days old)

✅ All data sources within acceptable freshness thresholds
---

## Data Loading

Load KTC valuation history for the past 90 days with player metadata

In [ ]:
# Query KTC historical values (past 90 days)
query = """
WITH latest_90_days AS (
    SELECT DISTINCT snapshot_date
    FROM main.stg_ktc__assets
    WHERE snapshot_date >= CURRENT_DATE - INTERVAL '90 days'
)
SELECT
    k.snapshot_date,
    k.player_name,
    p.position,
    p.age,
    p.nfl_team,
    k.ktc_value,
    k.overall_rank,
    k.position_rank
FROM main.stg_ktc__assets k
JOIN main.dim_player p ON k.player_id = p.player_id
WHERE k.snapshot_date IN (SELECT snapshot_date FROM latest_90_days)
  AND k.asset_type = 'player'
  AND k.league_format = '1qb'  -- 1QB league format
ORDER BY k.snapshot_date, k.ktc_value DESC
"""

ktc_history = conn.execute(query).fetchdf()
ktc_history['snapshot_date'] = pd.to_datetime(ktc_history['snapshot_date'])

print(f"Loaded {len(ktc_history)} KTC valuation records")
print(f"Date range: {ktc_history['snapshot_date'].min()} to {ktc_history['snapshot_date'].max()}")
print(f"Unique players: {ktc_history['player_name'].nunique()}")
ktc_history.head()

## Value Change Analysis

Calculate absolute and percentage changes in KTC values over the analysis period

In [ ]:
# Calculate value changes for each player
first_snapshot = ktc_history.groupby('player_name')['snapshot_date'].min()
last_snapshot = ktc_history.groupby('player_name')['snapshot_date'].max()

# Get first and last values
first_values = ktc_history.merge(
    first_snapshot.reset_index(),
    on=['player_name', 'snapshot_date']
)[['player_name', 'position', 'age', 'ktc_value']].rename(columns={'ktc_value': 'first_value'})

last_values = ktc_history.merge(
    last_snapshot.reset_index(),
    on=['player_name', 'snapshot_date']
)[['player_name', 'ktc_value', 'overall_rank']].rename(columns={'ktc_value': 'last_value'})

value_changes = first_values.merge(last_values, on='player_name')
value_changes['value_change'] = value_changes['last_value'] - value_changes['first_value']
value_changes['pct_change'] = (
    (value_changes['value_change'] / value_changes['first_value']) * 100
).round(2)

# Filter to significant players (value > 500)
value_changes = value_changes[value_changes['last_value'] >= 500]

print(f"Tracking {len(value_changes)} players with KTC value >= 500")

In [ ]:
# Biggest gainers (absolute value increase)
biggest_gainers = value_changes.nlargest(10, 'value_change')

print("📈 Top 10 Value Gainers (Past 90 Days):")
print("=" * 80)
print(biggest_gainers[[
    'player_name', 'position', 'age', 'first_value', 'last_value', 
    'value_change', 'pct_change'
]].to_string(index=False))

In [ ]:
# Biggest losers (absolute value decrease)
biggest_losers = value_changes.nsmallest(10, 'value_change')

print("📉 Top 10 Value Losers (Past 90 Days):")
print("=" * 80)
print(biggest_losers[[
    'player_name', 'position', 'age', 'first_value', 'last_value', 
    'value_change', 'pct_change'
]].to_string(index=False))

## Trend Visualizations

Visualize value trends for top players and identify patterns

In [ ]:
# Plot trends for top 10 overall players
top_10_players = value_changes.nsmallest(10, 'overall_rank')['player_name'].tolist()

top_10_history = ktc_history[ktc_history['player_name'].isin(top_10_players)]

fig, ax = plt.subplots(figsize=(14, 7))
sns.lineplot(
    data=top_10_history, 
    x='snapshot_date', 
    y='ktc_value', 
    hue='player_name',
    marker='o',
    linewidth=2
)

plt.title("KTC Value Trends - Top 10 Overall Players (Past 90 Days)", fontsize=14, fontweight='bold')
plt.xlabel("Date")
plt.ylabel("KTC Value")
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
plt.tight_layout()
plt.show()

In [ ]:
# Visualize biggest movers (top 5 gainers + top 5 losers)
top_gainers = value_changes.nlargest(5, 'value_change')['player_name'].tolist()
top_losers = value_changes.nsmallest(5, 'value_change')['player_name'].tolist()

movers = top_gainers + top_losers
movers_history = ktc_history[ktc_history['player_name'].isin(movers)]

fig, ax = plt.subplots(figsize=(14, 7))
sns.lineplot(
    data=movers_history, 
    x='snapshot_date', 
    y='ktc_value', 
    hue='player_name',
    marker='o',
    linewidth=2
)

plt.title("Biggest Market Movers (Top 5 Gainers + Top 5 Losers)", fontsize=14, fontweight='bold')
plt.xlabel("Date")
plt.ylabel("KTC Value")
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
plt.tight_layout()
plt.show()

## Positional Trend Analysis

Compare value stability across positions (RB volatility vs WR stability)

In [ ]:
# Calculate volatility by position
position_volatility = value_changes.groupby('position').agg({
    'value_change': ['mean', 'std'],
    'pct_change': ['mean', 'std'],
    'player_name': 'count'
}).round(2)

position_volatility.columns = [
    'Avg_Value_Change', 'Std_Value_Change',
    'Avg_Pct_Change', 'Std_Pct_Change',
    'Player_Count'
]

print("Position Volatility Analysis (90 Days):")
print("=" * 80)
print(position_volatility)

In [ ]:
# Visualize value change distribution by position
fig, ax = plt.subplots(figsize=(12, 6))

sns.boxplot(
    data=value_changes, 
    x='position', 
    y='pct_change',
    order=['QB', 'RB', 'WR', 'TE']
)

plt.axhline(y=0, color='red', linestyle='--', alpha=0.5, label='No Change')
plt.title("Value Change Distribution by Position (Past 90 Days)", fontsize=14, fontweight='bold')
plt.xlabel("Position")
plt.ylabel("% Change in KTC Value")
plt.legend()
plt.tight_layout()
plt.show()

## Trading Opportunity Identification

Find buy-low and sell-high candidates based on recent trends

In [ ]:
# Buy-low candidates: High value players with recent decline
buy_low = value_changes[
    (value_changes['last_value'] >= 2000) &  # Still high value
    (value_changes['pct_change'] < -15)  # Dropped >15%
].sort_values('pct_change')

if len(buy_low) > 0:
    print("💰 Buy-Low Candidates (High value + recent decline):")
    print("=" * 80)
    print(buy_low[[
        'player_name', 'position', 'age', 'last_value', 'value_change', 'pct_change'
    ]].to_string(index=False))
else:
    print("No buy-low candidates meeting criteria")

In [ ]:
# Sell-high candidates: Players with recent surge (potential overvaluation)
sell_high = value_changes[
    (value_changes['last_value'] >= 1500) &  # Decent value
    (value_changes['pct_change'] > 25)  # Increased >25%
].sort_values('pct_change', ascending=False)

if len(sell_high) > 0:
    print("📊 Sell-High Candidates (Recent surge, potential overvaluation):")
    print("=" * 80)
    print(sell_high[[
        'player_name', 'position', 'age', 'last_value', 'value_change', 'pct_change'
    ]].to_string(index=False))
else:
    print("No sell-high candidates meeting criteria")

## Conclusions

### Key Findings

1. **Market Movers**: [Summarize which players gained/lost the most value]
2. **Position Volatility**: [Note which positions showed most volatility - typically RBs]
3. **Buy-Low Opportunities**: [Highlight potential value buys]
4. **Sell-High Opportunities**: [Identify players at peak value to cash out]

### Recommendations

- **Immediate Trades**: Consider offers for sell-high candidates before market corrects
- **Monitoring**: Track buy-low candidates for further decline before acquiring
- **Position Strategy**: [Based on volatility, suggest position-specific strategies]
- **Next Analysis**: Re-run this analysis after next week's games to catch new trends